In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [11]:
df = ['SOXX', 'SOXL', 'TSM']
df = yf.download(df, start='2024-03-01')['Close']
print(df)

[*********************100%%**********************]  3 of 3 completed

Ticker           SOXL        SOXX         TSM
Date                                         
2024-03-01  48.330002  226.613327  133.899994
2024-03-04  49.770000  228.839996  138.259995
2024-03-05  46.840000  224.353333  134.970001
2024-03-06  50.220001  229.866669  141.570007
2024-03-07  55.320000  237.750000  149.199997
...               ...         ...         ...
2024-06-06  52.509998  240.740005  162.070007
2024-06-07  51.869999  240.020004  164.389999
2024-06-10  53.950001  243.479996  168.160004
2024-06-11  54.169998  243.369995  165.710007
2024-06-12  58.720001  250.229996  172.979996

[72 rows x 3 columns]


In [12]:
returns_df = df.pct_change().dropna()
returns_df

Ticker,SOXL,SOXX,TSM
Date,,,
2024-03-04,0.029795,0.009826,0.032562
2024-03-05,-0.058871,-0.019606,-0.023796
2024-03-06,0.072161,0.024574,0.048900
2024-03-07,0.101553,0.034295,0.053896
2024-03-08,-0.115148,-0.040463,-0.018968
...,...,...,...
2024-06-06,-0.018871,-0.008198,-0.005217
2024-06-07,-0.012188,-0.002991,0.014315
2024-06-10,0.040100,0.014415,0.022933


In [13]:
## Create a vector of weights which sum up to 1.
num_stocks = len(returns_df.columns) 
init_weights = [1/num_stocks] * num_stocks #vector of weights

## Calculate the annualised Expected Return of the portfolio
def getPortReturn(weights):
    exp_ret_portfolio = np.dot(np.transpose(weights), returns_df.mean()) * 250 #annualise crude method
    return exp_ret_portfolio

# Test the function with initial weights
annualized_return = getPortReturn(init_weights)
print("Annualized Portfolio Return: {:.2f}%".format(annualized_return * 100))


Annualized Portfolio Return: 80.24%


In [14]:
def getPortRisk(weights):
    '''Returns the annualised standard deviation of a k asset portfolio.'''

    # You don't need to recalculate the daily returns here if returns_df already exists.
    # Just use returns_df directly.
    
    # Calculate the covariance matrix of the asset returns.
    vcv = returns_df.cov() 
    
    # Calculate the variance of the portfolio's returns.
    # This is a dot product of weights with the covariance matrix, and then with weights again.
    var_p = np.dot(weights, np.dot(vcv, weights)) 
    
    # The standard deviation is the square root of variance.
    sd_p = np.sqrt(var_p)
    
    # To annualize, multiply by the square root of the number of trading days in a year.
    sd_p_annual = sd_p * np.sqrt(250)
    
    return sd_p_annual

# Example usage:
# Assuming you've already calculated and set 'init_weights' somewhere in your code.
portfolio_risk = getPortRisk(init_weights)
print("Annualized Portfolio Risk: {:.2f}%".format(portfolio_risk * 100))


Annualized Portfolio Risk: 48.48%


In [15]:
# Number of stocks should be based on the returns data, not the prices
num_stocks = len(returns_df.columns)  # Corrected to use returns_df
init_weights = [1 / num_stocks] * num_stocks  # Initialize weights

# Constraints for the optimization
bounds = tuple((0, 1) for _ in range(num_stocks))  # Bounds for each weight

# The sum of the weights must equal 1
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Perform the optimization to minimize portfolio risk
results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)
results  # Contains the output of the optimization process

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.290957476692221
       x: [ 0.000e+00  1.000e+00  0.000e+00]
     nit: 5
     jac: [ 8.457e-01  2.910e-01  2.964e-01]
    nfev: 20
    njev: 5

In [16]:
# Check total risk of the equal weighted portfolio
getPortRisk(init_weights)

0.48482527342851145

In [17]:
# Explore optimised weights
optimised_weights = pd.DataFrame(results['x'])
optimised_weights.index = returns_df.columns  # This should reference returns_df to match the stocks
optimised_weights.rename(columns={0: 'weights'}, inplace=True)  # Rename the column for clarity
optimised_weights['weights_rounded'] = optimised_weights['weights'].apply(lambda x: round(x, 3))

In [18]:
# Calculate individual annualized risks for each asset
individual_risks = np.std(returns_df) * np.sqrt(250)

# Add individual risks to the optimised_weights DataFrame
optimised_weights['individual_risks'] = individual_risks

# Display the optimised_weights DataFrame with the new 'individual_risks' column
optimised_weights

/Users/aaryanbhatia/Library/Python/3.11/lib/python/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


,weights,weights_rounded,individual_risks
Ticker,,,
SOXL,0.0,0.0,0.841131
SOXX,1.0,1.0,0.288901
TSM,0.0,0.0,0.375258


In [19]:
def get_correlation_matrix():
     return np.round(returns_df.corr(), 2)
 
 
get_correlation_matrix()

Ticker,SOXL,SOXX,TSM
Ticker,,,
SOXL,1.00,1.00,0.79
SOXX,1.00,1.00,0.78
TSM,0.79,0.78,1.00


In [28]:
# Define the maximum risk level you are willing to accept
bounds = tuple((0, 1) for i in range (num_stocks)) # for loop, create tuple of 0, 1 for every stock
max_risk = 0.3  # for example, 15%

def objective(weights):
    # Objective function to maximize the return (minimize the negative return)
    return -getPortReturn(weights)

def risk_constraint(weights):
    # Constraint function to ensure the risk does not exceed max_risk
    return max_risk - getPortRisk(weights)

# Define constraints for the optimization
cons = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Weights must sum up to 1
        {'type': 'ineq', 'fun': risk_constraint}]  # Risk must not exceed max_risk

# Run the optimization
results = minimize(fun=objective, x0=init_weights, bounds=bounds, constraints=cons)

# Check results
if results.success:
    optimized_weights = results.x
    print("Optimized weights:", optimized_weights)
else:
    print("Optimization failed:", results.message)

Optimized weights: [8.66996843e-14 7.15922160e-01 2.84077840e-01]


In [29]:
optimized_weights = results.x

# Calculate the portfolio risk with the optimized weights
optimized_risk = getPortRisk(optimized_weights)

# Calculate the annualized return with the optimized weights
optimized_return = getPortReturn(optimized_weights)

# Calculate individual risks (standard deviations)
individual_risks = np.std(returns_df) * np.sqrt(250)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Optimized Weights': optimized_weights,
    'Individual Risks': individual_risks
}, index=returns_df.columns)

# Add a column for rounded weights
results_df['Weights Rounded'] = results_df['Optimized Weights'].apply(lambda x: round(x, 3))

# Add the total optimized return and risk at the end of the DataFrame
total_metrics = pd.DataFrame({
    'Optimized Weights': ['Total Portfolio'],
    'Individual Risks': [optimized_risk],
    'Weights Rounded': [optimized_return]
})

# Combine the individual asset data with the total metrics
results_df = pd.concat([results_df, total_metrics])

# Display the results DataFrame
results_df

/Users/aaryanbhatia/Library/Python/3.11/lib/python/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


,Optimized Weights,Individual Risks,Weights Rounded
SOXL,0.0,0.841131,0.000000
SOXX,0.715922,0.288901,0.716000
TSM,0.284078,0.375258,0.284000
0,Total Portfolio,0.300000,0.556477


In [30]:
# Assuming 'results' contains the successful optimization output
optimized_weights = results['x']

# Calculate the portfolio return using the optimized weights
portfolio_return = getPortReturn(optimized_weights)
print("Optimized Portfolio Return: {:.2f}%".format(portfolio_return * 100))

# Calculate the portfolio risk using the optimized weights
portfolio_risk = getPortRisk(optimized_weights)
print("Optimized Portfolio Risk: {:.2f}%".format(portfolio_risk * 100))

Optimized Portfolio Return: 55.65%
Optimized Portfolio Risk: 30.00%
